In [81]:
import time
import multiprocessing
import threading
import math
from queue import Queue
import concurrent.futures

In [86]:
'''
1. Discuss the scenarios where multithreading is preferable to multiprocessing and scenarios where
multiprocessing is a better choice.
Answer:
01.MultiThreading is the generally preferred for the tasks that are I/O bound (such as network or file I/O),
    where tasks mostly wait for external data and CPU usage is low.
    Threads share memory space and can communicate easily, making them ideal for lightweight, concurrent tasks.
    Due to the Global Interpreter Lock (GIL) in python, only one thread can execute python bytecode at a time,
    limiting CPU-bound performance benefits

Scenarios and Examples:

# MultiThreading scenario 1 / Web scraper for multiple sites (I/O bound tasks)
    MultiThreading is suitable here (In this example) as we fetch data from multiple sites, and each thread can handle a different
    request without much CPU load.
'''
#
start = time.perf_counter()

url = [
    "http://www.example.com",
    "http://www.example.org",
    "http://www.example.net"
]

data = ["file_1", "file_2", "file_3"]

import urllib.request
import concurrent.futures

def download_file(url, file):
    try:
        response = urllib.request.urlopen(url)
        if response.status == 200:  # 200 means the URL is accessible
            urllib.request.urlretrieve(url, file)
            print(f"Downloaded {file} from {url}")
        else:
            print(f"Failed to download {file} from {url} - Status: {response.status}")
    except Exception as e:
        print(f"Error downloading {file} from {url}: {e}")

with concurrent.futures.ThreadPoolExecutor() as e:
    e.map(download_file, url, data)


'''
Explanation:
    Each thread fetched data from URL simultaneously, speeding up network-bound tasks by overlapping wait times for
    each site.


MultiThreading scenario 2: Live score and updating dashboard (low cpu tasks)
    This thread examples are used to simulate real time updates for multiple scores, which dont require high cpu usage
    '''
lock = threading.Lock()
def update_score(score_name):
    for i in range(5):
        with lock:  # Locking to ensure only one thread prints at a time
            print(f"Updating {score_name} score")
        time.sleep(1)  # Delay of 1 second

# simulating 2 updates
threads = [
    threading.Thread(target= update_score, args= ("Football",)),
    threading.Thread(target=update_score, args=("Basketball",))
]

# starting threads
for thread in threads:
    thread.start()

# waiting for the first process to complete
for thread in threads:
    thread.join()
print("All score updates completed")
'''
Explanation:
    Each thread handles score updates for different sports in parallel making it efficient for scenarios with low
    computational load.

02.MultiProcessing is better for CPU-bound tasks that required significant processing power, each process has its,
    own memory space, allowing it to run independently, which is efficient for tasks like data processing, image
    manipulation, and calculations that can benefit from parallel execution.
    In python, multiprocessing avoids the GIL, so each process can fully utilize a Cpu core.

Multiprocessing scenerio 1 : Image processing (cpu bound tasks)
    image processing requires substantial CPU power, making multi processing ideal for this tasks, as each process
    can use a seperate core
Scenario / Example 1
    '''
import multiprocessing
import time

def process_image(image_id):
    print(f"Processing image {image_id}")
    time.sleep(1)
    # processing the image
    print(f"Image {image_id} processed")


if __name__ == "__main__":
    # creating process for each image
    processes = [multiprocessing.Process(target=process_image, args=(i,)) for i in range(1, 4)]

    # Starting process
    for process in processes:
        process.start()

    # will wait for all the above process
    for process in processes:
        process.join()
'''
Explanation:
    Each process works on a separate image, enabling faster processing by leveraging multiple cores

Mulitpprocessing : Scenario 2 Parrel Calculations (Independent Process)
    Multi_processing is beneficial here, as each process performs calculations independently,
    fully utilizing multiple CPU cores.

Example:
'''
def calculate_square(number):
    square =  number*number
    time.sleep(0.5)
    print(f"The square of {number} is: {square}")


numbers = [23, 26 , 35, 64]

processes = [multiprocessing.Process(target=calculate_square, args=(number,)) for number in numbers]

# starting and joining each process
for process in processes:
    process.start()
for process in processes:
    process.join()

print("All processes completed")
end = time.perf_counter()

print(f"{round(end-start, 2)} seconds taken by the process to execute")

'''
Explanation
    Each process independently calculates the squares root of a number list, making it
    efficient for tasks that do not require inter-process communication.

Insights:
    MultiThreading is best for I/O bound tasks like network requests or real-time
    updates, where threads can efficiently wait for the responses without overloading the CPU

    MultiProcessing is best for CPU bound tasks like complex calculations or image processing,
    allowing each core to handle separate tasks in parallel for high efficient.

'''
print()

Downloaded file_3 from http://www.example.netDownloaded file_2 from http://www.example.org

Downloaded file_1 from http://www.example.com
Updating Football score
Updating Basketball score
Updating Football score
Updating Basketball score
Updating Football score
Updating Basketball score
Updating Football score
Updating Basketball score
Updating Football score
Updating Basketball score
All score updates completed
Processing image 1
Processing image 2
Processing image 3
Image 1 processed
Image 2 processed
Image 3 processed
The square of 23 is: 529
The square of 26 is: 676
The square of 35 is: 1225
The square of 64 is: 4096
All processes completed
6.74 seconds taken by the process to execute



In [87]:
'''
Q2. Describe what a process pool is and how it helps in managing multiple processes efficiently.
Answer:
    A process pool is a programming abstraction that allows for the efficient management of multiple processes
    in parallel.
    It provides a "pool" of worker processes that can perform tasks concurrently, enabling easy distribution and
    management of tasks across several processes.

How a Process Pool works?

    In python, multiprocessing.Pool is often used to create a process pool, the pool has a fixed number of worker
    processes, defined when the pool is created, and each worker process completes its tasks , it becomes available to
    handle the next task, This ensures that no extra processes are created unnecessarily,
    managing resource efficiently

Benefits of a Process Pool?

    1.Resource management: It restricts the number of processes, preventing system overload by limiting CPU and
    memory usage
    2.Task management: Instead of manually creating and closing processes, the pool automatically assigns tasks to
    available processes and reuse them for new tasks
    3.Concurrency: Since each process runs independently on its own CPU core, a process pool makes it easy to perform
    multiple tasks in parallel.
    4.Simplicity: Using a process pool abstracts away the need to handle inter-process communication directly
    simplifying code

Example of Process pool to calculate the square of multiple numbers concurrently"
'''


# function that calculate squares
def square(x):
    return x * x


start = time.perf_counter()

# list of numbers to be calculated
numbers = [13632, 23432, 312131, 456575, 54543, 689654, 9]

# creating process pool with three workers
with multiprocessing.Pool(3) as pool:
        # maping numbers list to the square function using the pool
    result = pool.map(square, numbers)

    # printing result
    print("Squares: ", result)

    # calculating and printing the execution time
    end = time.perf_counter()
    print(f"Time taken to execute the process: {round(end - start, 2)} seconds.")

'''
Explanation:
    In the example, the pool of three processes will concurrently process the numbers in numbers, calculating their
    squares.
    The pool automatically handles the task assignment, maximizing efficiency by keeping all processes busy without
    creating excessive overhead.

Insights:
    A process pool is a group of worker processes that helps run tasks in parallel speeding up performance, instead of
    creating a new process for each task, it reuses these workers, which saves time and system resources.
    This setup is useful when you have multiple tasks that can run independently at the same time, making it easier and
    more efficient to manage
'''
print()

Squares:  [185831424, 549058624, 97425761161, 208460730625, 2974938849, 475622639716, 81]
Time taken to execute the process: 0.05 seconds.



In [88]:
'''
Q3. Explain what multiprocessing is and why it is used in Python programs.
Answer:
    Multiprocessing is a method that allows python programs to run multiple process at the same time, utilizing multiple
    CPU cores.
    This is helpful because python's Global interpreter Lock (GIL) restricts a single program thread from executing on more than one core.
    By using multiprocessing, each process can bypass the GIL, allowing parts of a program to execute parallel, This is especially beneficial in
    CPU-intensive tasks, like data processing or complex calculations, as it boosts speed and overall program efficiency
'''
def factorial(x):
    return math.factorial(x)

# numbers to calculate factorial
numbers = [1, 2, 3, 4, 5, 8, 9, 45]

with multiprocessing.Pool(3) as pool:
    start = time.perf_counter()
    factorials = pool.map(factorial, numbers)
    end = time.perf_counter()
print("Factorial numbers are", factorials)
print(f"{round(end-start)} second has taken by the process execution")

'''
Insights:
    Benefits of Multiprocessing: Calculating large factorial is CPU-intensive , by using multiprocessing, each
    factorial calcullation runs in parallel on a separate core, resulting in faster total computation time.

    Real_world use : This method is particularly useful in applications where multiple large calculations need to
    be perform simultaneously, such as scientific computing or big data analysis
'''
print()

Factorial numbers are [1, 2, 6, 24, 120, 40320, 362880, 119622220865480194561963161495657715064383733760000000000]
0 second has taken by the process execution



In [89]:
'''
Q4. Write a Python program using multithreading where one thread adds numbers to a list, and another
thread removes numbers from the list. Implement a mechanism to avoid race conditions using
threading.Lock.
'''
start = time.perf_counter()
number_list = []
print("Empty list", number_list)

lock = threading.Lock()
def adding_numbers(number):
    with lock:
        for i in number:
            number_list.append(i)


def removing_numbers():
    with lock:
        number_list.clear()


t1 = threading.Thread(target = adding_numbers, args = ([1, 2, 3, 4, 5, 6, 7],))
t1.start()
t1.join()
print("Added number to the list", number_list)


t2 = threading.Thread(target = removing_numbers)
t2.start()
t2.join()
print("Removed numbers from the list:", number_list)
end = time.perf_counter()
print(f"Time taken to execute the process: {round(end - start, 2)} seconds.")
'''
insight:
    In this code, multi threading is used to add and remove numbers from the list, threading.Lock() helps avoid race
    condition by preventing one thread's actions from interfering with another.
    By using the lock, we ensure that when one thread is modifying the list, the other cannot accessit.
    This sync approach helps threads handle data safely and accurately, which is essential in concurrent programming
  '''
print()

Empty list []
Added number to the list [1, 2, 3, 4, 5, 6, 7]
Removed numbers from the list: []
Time taken to execute the process: 0.01 seconds.



In [90]:
'''
Q5. Describe the methods and tools available in Python for safely sharing data between threads and
processes.
Answer:
    The methods and tool available in python for safety sharing data between threads and processes are,
    as following:

01.Threading Locks(threading.lock):
    locks are essential to prevent race condition, which occur when multiple threads access shared data simultaneously,
    potentially corrupting it
Example:
'''
start = time.perf_counter()
counter = 0
counter_lock = threading.Lock() # locking the counter for specific thread

def increment_shared_data(x):
    global counter
    with counter_lock:
        counter = counter + 1
        print(f"Thread {x} increamented shared counter to: {counter}")
        time.sleep(1)

threads = [threading.Thread(target= increment_shared_data, args= (i,))for i in [1, 2, 3, 4, 5,6, 7]]

for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

end = time.perf_counter()
print(f"{round(end-start, 2)} second has been taken to execute this process.")
print()
'''
This code ensures only one thread increments shared_data at a time.

02.Queues(queue for threads, multiprocessing.Queue for processes)
    Queues manage data in a "first in , first out (FIFO) fashion and use an interlock to ensures
    thread/process safety.

Example:
'''
q = Queue()

# filling queues with urls
urls = [
    "https://example.com/page1",
    "https://example.com/page2",
    "https://example.com/page1"
]
for url in urls:
    q.put(url)

def url():
    while not q.empty():
        url = q.get()
        print(f"Processing url:{url}")
        time.sleep(1)
        q.task_done()

# start multiple threads
threads = [threading.Thread(target=url) for i in range(3)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

end = time.perf_counter()

print("All urls Crawled")
print(f"{round(end-start, 2)} second has taken to process both the execution.")
print()
'''
03. Event object for production line simulation.
    A production line where one worker must wait for parts to be manufactured before assembling them.

Example:
'''
parts_ready = threading.Event()

def manufacture_parts():
    print("Manufacturing parts..")
    time.sleep(1)
    parts_ready.set()# will notify when the parts are ready

def assemble_parts():
    print("Waiting for the parts to be ready..")
    parts_ready.wait() # waiting till the manufacturing is done
    print("Assembling the parts..")

threads = [
    threading.Thread(target= manufacture_parts),
    threading.Thread(target=assemble_parts)
]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

end = time.perf_counter()

print(f"{round(end-start,2)} second has been taken to complete all three process execution.")
print()
'''
04.Shared memory with counter:
    Tracking the number of items produced in a factory, updated by multiple processes.

Example:
'''
item_count = multiprocessing.Value("i", 0) #Shared integer value , "i" denoted integer type

def produce_items():
    for i in range(100):
        with item_count.get_lock(): # for automatic update
            item_count.value += 1

processes = [multiprocessing.Process(target = produce_items) for i in range(5)]
for process in processes:
    process.start()
for process in processes:
    process.join()

print("Total item produced is: ", item_count.value)
end = time.perf_counter()

print(f"{round(end-start,2)} second has been taken to complete all four process execution.")
print()
'''
05.Manager list with online order fulfillment
    Multiple processes updating a shared list of completed orders.

Example:
'''
def complete_order(order_list, order_id):
    order_list.append(f"Order {order_id} completed")

with multiprocessing.Manager() as manager:
    completed_orders = manager.list() #shared list
    processes = [
        multiprocessing.Process(target=complete_order, args=(completed_orders, i)) for i in range(1, 7)
    ]
    for process in processes:
        process.start()
    for process in processes:
        process.join()
    print("Completed Orders", list(completed_orders))

end = time.perf_counter()
print()
print(f"{round(end-start,2)} second has been taken to complete all five process execution.")

'''
Insights:

    Each of these examples demonstrates common real world applications for managing shared resources across
    threads and processes, whether for counter increment, web scrapping, production lines, item counters , or
    order processing, these approaches ensure data safety while optimizing performance.
    '''
print()

Thread 1 increamented shared counter to: 1
Thread 2 increamented shared counter to: 2
Thread 3 increamented shared counter to: 3
Thread 4 increamented shared counter to: 4
Thread 5 increamented shared counter to: 5
Thread 6 increamented shared counter to: 6
Thread 7 increamented shared counter to: 7
7.02 second has been taken to execute this process.

Processing url:https://example.com/page1
Processing url:https://example.com/page2
Processing url:https://example.com/page1
All urls Crawled
8.02 second has taken to process both the execution.

Manufacturing parts..
Waiting for the parts to be ready..
Assembling the parts..
9.03 second has been taken to complete all three process execution.

Total item produced is:  500
9.13 second has been taken to complete all four process execution.

Completed Orders ['Order 1 completed', 'Order 2 completed', 'Order 4 completed', 'Order 3 completed', 'Order 6 completed', 'Order 5 completed']

9.3 second has been taken to complete all five process execu

In [91]:
'''
Q6. Discuss why it’s crucial to handle exceptions in concurrent programs and the techniques available for
doing so.
Answer:
    Handling exceptions in concurrent programs i crucial because, without it, errors in one thread or process could
        disrupt or corrupt the entire program.
    Concurrent tasks often share resources, and an unhandled exception might lead to data inconsistencies,
        deadlocks, or crashes that are hard to debug, Ensuring exception safety helps maintain program stability,
            data integrity and smooth operation, especially in complex, long running systems

Techniques for handling Exceptions in Concurrent programs
    1.Try except blocks in threads/processes:
        Wrap the code in each thread or process with a try-except block to handle exceptions locally, this ensures
        that if a particular thread or process encounters an issue, it wont crash the entire program.
    Example:
'''
start = time.perf_counter()


def worker ():
    try:
        result = 10/0  # suspecious code
        # cause a ZeroDivisionError
    except Exception as e:
        print(f"The error in worker using try exception: {e}")

thread = threading.Thread(target=worker)
thread.start()
thread.join()

'''
02.Using Thread/Process return values
    In concurrent programming libraries like concurrent.futures, you can use futures to get return values from the
        threads or processes.
    These futures provide and exception() method to check if an exception occurred.

    Example:
'''


def tasks ():
    return 10 / "10"  #suspecious code


with concurrent.futures.ThreadPoolExecutor() as e:
    future = e.submit(tasks)
    try:
        result = future.result()
    except Exception as e:
        print(f"Error using return value  : {e}")

'''
03.Using Queues to pass Exceptions:
    In complex applications, you can use a queue to pass exceptions from threads or processes back to the main thread,
        which can then handle them or log them as necessary.
    Example:
'''

def worker(error_queue):
    try:
        result = 7/0 #suspecious code
    except Exception as e:
        error_queue.put(e) #Sending error to main thread

q = Queue()


thread = threading.Thread(target=worker, args=(q,))
thread.start()
thread.join()

if not q.empty():
    error = q.get()
    print(f"Error in worker using queue {error}")

'''
Timeouts for task completion
    Set timeouts on tasks to avoid indefinitely hanging due to exceptions like deadlocks or resource locks,
        especially in multiprocessing.

    Example:
'''

def long_running_tasks():
    time.sleep(1)
    return "Done"
with concurrent.futures.ThreadPoolExecutor( ) as e:
    future = e.submit(long_running_tasks)
    try:
        result = future.result(timeout=6)
    except TimeoutError:
        print(f"Tasks took to long and terminated")
end = time.perf_counter()
print(f"It took {round(end-start, 2)} second to complete the execution process")

'''
Insights:
    Effective exception handling in concurrent programs prevents system wide failures and ensures data safely
     across threads and processes.
     Techniques like local try except handling using features to monitor tasks states or sharing exceptions via queues
     help isolate errors, making it easier to debug and maintain robust applications
    '''
print()

The error in worker using try exception: division by zero
Error using return value  : unsupported operand type(s) for /: 'int' and 'str'
Error in worker using queue division by zero
It took 1.01 second to complete the execution process



In [92]:
'''
Q7. Create a program that uses a thread pool to calculate the factorial of numbers from 1 to 10 concurrently.
Use concurrent.futures.ThreadPoolExecutor to manage the threads.
Answer_code:
    The code to perform factorial with concurrent.futures is as following:
'''

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9 ,10]
with concurrent.futures.ThreadPoolExecutor(3) as pool:
    start = time.perf_counter()
    factorials = list(pool.map(lambda x : math.factorial(x), numbers))
    end = time.perf_counter()

    time.sleep(0.1)
    print("Factorial numbers are", factorials)
    print(f"{round(end - start, 2)} second has taken by the process execution")

'''
Explanation:
    1.Concurrent.futures.ThreadPoolExecutor(3) will work on 3 processor
    2.factorials will map with pool.map() using lambda function
    3.Tracking the execution time of the process
    3. Printing the factorial numbers and execution time

Insights:
    This code return the factorial numbers given in numbers variable, using pool.map() and
    lambda function inside it, and printing the factorial numbers and the execution time.
'''
print()

Factorial numbers are [1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]
0.01 second has taken by the process execution



In [93]:
'''
Q8. Create a Python program that uses multiprocessing.Pool to compute the square of numbers from 1 to 10 in
parallel. Measure the time taken to perform this computation using a pool of different sizes (e.g., 2, 4, 8
processes).
'''
def square(x):
    return x*x

# list of numbers between 1 to 10
numbers = [1, 2, 3, 4 ,5 ,6 ,7 ,8, 9, 10]

# using multiprocessing with two worker
with multiprocessing.Pool(2) as pool:
    start = time.perf_counter()

        # mapping with pool.map()
    squares = pool.map(square, numbers)
    print(f"square of numbers given between 1 to 10 is:\n {squares}")
    end = time.perf_counter()
    print(f"Two Processor takes {round(end-start, 3)} seconds to complete the execution")

print()

# using multiprocessing with four worker
with multiprocessing.Pool(4) as pool:
    start = time.perf_counter()

    # mapping with pool.map()
    squares = list(pool.map(square, numbers))
    print(f"Another squaring the numbers with another more processor:\n {squares}")
    end = time.perf_counter()
    print(f"Four Processor takes {round(end-start, 3)} seconds to complete the execution")

print()
# using multiprocessing with eight worker
with multiprocessing.Pool(8) as pool:
    start = time.perf_counter()

        # mapping with pool.map()
    squares = list(pool.map(square, numbers))
    print(f"Another squaring the numbers with another more processor:\n {squares}")
    end = time.perf_counter()
    print(f"Eight Processor takes {round(end-start, 3)} seconds to complete the execution.")

'''
Explanation:
    1.Function Definition: square(x) calculates the square of a number.
    2.Numbers List: A list of numbers from 1 to 10 is created.
    3.Multiprocessing Pool: Different pool sizes (2, 4, 8) are used to compute squares in parallel.
    4.Time Measurement: time.perf_counter() tracks how long each computation takes

Insights:
    The output shows minimal differences in execution time across different processor counts due to the small dataset size.
    The results confirm that multiprocessing can efficiently handle tasks even with small computations, but the benefits
    may be more pronounced with larger datasets.
'''
print()

square of numbers given between 1 to 10 is:
 [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Two Processor takes 0.01 seconds to complete the execution

Another squaring the numbers with another more processor:
 [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Four Processor takes 0.007 seconds to complete the execution

Another squaring the numbers with another more processor:
 [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Eight Processor takes 0.006 seconds to complete the execution.

